In [1]:
# Dependencies
import time
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
import os
import re

game_info_df=pd.read_csv("bgg_csv/boardgames2_06022021.csv")
ranking_df=pd.read_csv("bgg_csv/2021-05-29_game_id_rankings.csv")

In [2]:
 # URL of page to be scraped
url = 'https://boardgamegeek.com/blog/1/boardgamegeek-news'
# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'html.parser')
news_titles = soup.find(class_='blog_post')
# find the news title and image
news_title = news_titles.find(class_='post_title').text
featured_image_url = soup.find(class_="post-img").a.img['src']
print(f'news_title: {news_title}')
print(f"featured_image_url: {featured_image_url}")
news_df=pd.DataFrame([{"news_title":news_title, "featured_image_url":featured_image_url}])
news_df.to_json("data/news.json")
news_df.to_json("../without_flask/data/news.json")

news_title: 
Zip Back and Forth in TRAILS, and Hunt Humans to Satisfy The Hunger

featured_image_url: https://cf.geekdo-images.com/alEgV88cpqFtaHAklI6lbw__medium/img/7u112Ar9dnLKQ1WKAa9yYPjiW0w=/fit-in/500x500/filters:no_upscale():strip_icc()/pic6214088.jpg


In [3]:
# get the top 200 rankings
ranking_df.drop_duplicates(subset=['BoardGameRank'], inplace=True)
ranking_df.fillna(0, inplace=True)
ranking_int_df=ranking_df.astype('int64')
ranking_int_df['BoardGameRank'] = ranking_int_df.BoardGameRank.astype(str)
ranking_200_df=ranking_int_df.head(200).copy()
ranking_200_df.set_index('BoardGameRank', inplace=True)
ranking_200_dict=ranking_200_df.to_dict()
ranking_200_df.to_json('data/ranking.json')
ranking_200_df.to_json('../without_flask/data/ranking.json')
# ranking_200_dict

In [4]:
# check if any game in the top 200 list is not the 20k game info list
game_id_list=game_info_df['objectid']
game_20k_list=[*game_id_list]
game_20k_set=set(game_20k_list)

game_ids=[]
for i in range(200):
    game_id=ranking_200_df.iloc[i].unique()
    game_ids.extend(game_id)
unique_game_ids=set(game_ids)
unique_game_ids.remove(0)

game_out=0
for game_id in unique_game_ids:
    if game_id not in game_20k_set:
        print(f'{game_id} not found')
        game_out+=1
print(f'there are/is {game_out} game(s) from top 200 games that not cover in the 20k game info')

there are/is 0 game(s) from top 200 games that not cover in the 20k game info


In [5]:
# add a column "is_top200" to game_info_df
is_top200_list=[]
for game_id in game_info_df['objectid']:
    if game_id in unique_game_ids:
        is_top200_list.append(True)
    else:
        is_top200_list.append(False)
game_info_df['is_top200']=is_top200_list

In [6]:
# convert unicode to printable format
a=game_info_df['name']
kk=[]
for b in a:
    b=re.sub('\s\s+', ' ', b)
    try:
        c=(b.encode('utf-8').decode('unicode-escape'))
        kk.append(c)
    except:
        x=b.replace('\\u','/u').replace('\\', "").replace('/u','\\u')
        c=(x.encode('utf-8').decode('unicode-escape'))
        kk.append(c)
game_info_df['game_name']=kk

<ipython-input-6-e81dacafd4eb>:7: DeprecationWarning: invalid escape sequence '\/'
  c=(b.encode('utf-8').decode('unicode-escape'))
<ipython-input-6-e81dacafd4eb>:7: DeprecationWarning: invalid escape sequence '\W'
  c=(b.encode('utf-8').decode('unicode-escape'))
<ipython-input-6-e81dacafd4eb>:7: DeprecationWarning: invalid escape sequence '\T'
  c=(b.encode('utf-8').decode('unicode-escape'))
<ipython-input-6-e81dacafd4eb>:7: DeprecationWarning: invalid escape sequence '\B'
  c=(b.encode('utf-8').decode('unicode-escape'))
<ipython-input-6-e81dacafd4eb>:7: DeprecationWarning: invalid escape sequence '\A'
  c=(b.encode('utf-8').decode('unicode-escape'))
<ipython-input-6-e81dacafd4eb>:7: DeprecationWarning: invalid escape sequence '\O'
  c=(b.encode('utf-8').decode('unicode-escape'))
<ipython-input-6-e81dacafd4eb>:7: DeprecationWarning: invalid escape sequence '\D'
  c=(b.encode('utf-8').decode('unicode-escape'))
<ipython-input-6-e81dacafd4eb>:7: DeprecationWarning: invalid escape sequenc

In [7]:
# parse the string into list of strings
bg_cat=game_info_df['boardgamecategory']
game_info_df['boardgamecategory']=[cc[1:-1].replace("'",'').split(",") for cc in bg_cat]
bg_pub=game_info_df['boardgamepublisher']
game_info_df['boardgamepublisher']=[cc[1:-1].replace("'",'').split(",") for cc in bg_pub]
bg_mec=game_info_df['boardgamemechanic']
game_info_df['boardgamemechanic']=[cc[1:-1].replace("'",'').split(",") for cc in bg_mec]

In [8]:
# get only the needed columns
game_info_selected_df=game_info_df[['objectid', 'game_name', 'description', 'yearpublished','is_top200',
                                    'average','numplays','maxplaytime','minage', 'languagedependence',
                                    'minplayers','maxplayers', 'minplaytime',  
                                    'news', 'blogs', 'weblink','podcast', 
                                    'boardgamepublisher', 'boardgamecategory', 'boardgamemechanic','gamelink']].copy()
# drop the duplicates based on objectid
game_info_selected_df.drop_duplicates(subset=['objectid'], inplace=True)
game_info_selected_df['objectid'] = game_info_selected_df.objectid.astype(str)
game_info_selected_df=game_info_selected_df.sort_values(by=['game_name']).reset_index(drop=True)
game_info_selected_df.set_index('objectid', inplace=True)
game_info_selected_df.T.to_json('data/game_info.json')
game_info_selected_df.T.to_json('../without_flask/data/game_info.json')
game_info_dict=game_info_selected_df.to_dict('index')
# game_info_dict

In [9]:
boardgame_data={"news_title":news_title, "featured_image_url":featured_image_url,"ranking":ranking_200_dict,"game_info":game_info_dict}
boardgame_data

{'news_title': '\nZip Back and Forth in TRAILS, and Hunt Humans to Satisfy The Hunger\n',
 'featured_image_url': 'https://cf.geekdo-images.com/alEgV88cpqFtaHAklI6lbw__medium/img/7u112Ar9dnLKQ1WKAa9yYPjiW0w=/fit-in/500x500/filters:no_upscale():strip_icc()/pic6214088.jpg',
 'ranking': {'2018-10-06': {'1': 174430,
   '2': 161936,
   '3': 182028,
   '4': 167791,
   '5': 12333,
   '6': 187645,
   '7': 169786,
   '8': 220308,
   '9': 120677,
   '10': 193738,
   '11': 84876,
   '12': 173346,
   '13': 180263,
   '14': 115746,
   '15': 3076,
   '16': 102794,
   '17': 205637,
   '18': 183394,
   '19': 31260,
   '20': 233078,
   '21': 96848,
   '22': 124361,
   '23': 205059,
   '24': 164928,
   '25': 170216,
   '26': 28720,
   '27': 209010,
   '28': 175914,
   '29': 164153,
   '30': 177736,
   '31': 25613,
   '32': 162886,
   '33': 221107,
   '34': 2651,
   '35': 55690,
   '36': 72125,
   '37': 230802,
   '38': 126163,
   '39': 35677,
   '40': 121921,
   '41': 171623,
   '42': 124742,
   '43': 68